<!--<badge>--><a href="https://colab.research.google.com/github/huggingface/workshops/blob/main/luzern-university/06-stable-diffusion-gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

# Creating a Diffusers Demo with Gradio

**Learning goals:** The goal of this tutorial is to learn How To

1. Build a quick demo for your machine learning model in Python using the `gradio` library
2. Host the demos for free with Hugging Face Spaces

**Duration**: 20-40
 minutes

**Prerequisites:** Knowledge of Python and basic familiarity with machine learning

All of these steps can be done for free! All you need is an Internet browser and a place where you can write Python 👩‍💻

## Why Demos?

**Demos** of machine learning models are an increasingly important part of machine learning _courses_ and _conferences_. Demos allow:

* model developers to easily **present** their work to a wide audience
* increase **reproducibility** of machine learning research
* diverse users to more easily **identify and debug** failure points of models


As a quick example of what we would like to build, check out the [Keras Org on Hugging Face](https://huggingface.co/keras-io), which includes a description card and a collection of Models and Spaces built by Keras community. Any Space can be opened in your browser and you can use the model immediately, as shown here: 

![](https://i.ibb.co/7y6DGjB/ezgif-5-cc52b7e590.gif)




## Setup

If you're running this notebook on Google Colab or locally, you'll need a few dependencies installed. You can install them with `pip` as follows:

In [ ]:
#! pip install diffusers transformers ftfy gradio

To be able to access the Stable Diffusion checkpoints from the Hugging Face Hub, you'll need to store your authentication token from the Hugging Face website. Sign up [here](https://huggingface.co/join) if you haven't already, then execute the following cell and input your API token:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Finally, let's set the device (CPU or GPU) to run our pipelines on:

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if device == "cuda" else None

## Building a demo with Gradio

`gradio` is a handy Python library that lets you build web demos simply by specifying the list of input and output **components** expected by your machine learning model. 

What do I mean by input and output components? Gradio comes with a bunch of predefined components for different kinds of machine learning models. Here are some examples:

* For an **image classifier**, the expected input type is an `Image` and the output type is a `Label`. 
* For a **speech recognition model**, the expected input component is an `Microphone` (which lets users record from the browser) or `Audio` (which lets users drag-and-drop audio files), while the output type is `Text`. 
* For a **question answering model**, we expect **2 inputs**: [`Text`, `Text`], one textbox for the paragraph and one for the question, and the output type is a single `Text` corresponding to the answer. 

You get the idea... (for all of the supported components, [see the docs](https://gradio.app/docs/))

In addition to the input and output types, Gradio expects a third parameter, which is the prediction function itself. This parameter can be ***any* regular Python function** that takes in parameter(s) corresponding to the input component(s) and returns value(s) corresponding to the output component(s)

Enough words. Let's see some code!

First, let's load the `StableDiffusionPipeline` and check it can generate an image:

In [ ]:
from diffusers import StableDiffusionPipeline

model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id, revision="fp16", torch_dtype=torch_dtype
).to(device)
prompt = "a photograph of an astronaut riding a horse"

outputs = pipe(prompt)
outputs.images[0]

Next, we need to implement a function that takes a text prompt and returns an image:

In [7]:
def predict(prompt):
    return pipe(prompt).images[0]

predict(prompt)

The final step is to implement a simple interface:

In [ ]:
import gradio as gr

gradio_ui = gr.Interface(
    fn=predict,
    title="Stable Diffusion Demo",
    description="Enter a description of an image you'd like to generate!",
    inputs=[
        gr.Textbox(lines=2, label="Paste some text here"),
    ],
    outputs=["image"],
    examples=[["a photograph of an astronaut riding a horse"]],
)

gradio_ui.launch(debug=True)

Running the code above should produce a simple GUI inside this notebook allowing you to type example inputs and see the output returned by your function.

Notice that we define an `Interface` using the 3 ingredients mentioned earlier:
* A function
* Input component(s)
* Output component(s)

This is a simple example for images, but the same principle holds true for any other kind of data type.

## Host the Demo (for free) on Hugging Face Spaces

Once you made a Gradio demo, you can host it permanently on Hugging Spaces very easily:

Here are the steps to that (shown in the GIF below):

A. First, create a Hugging Face account if you do not already have one, by visiting https://huggingface.co/ and clicking "Sign Up"

B. Once you are logged in, click on your profile picture and then click on "New Space" underneath it to get to this page: https://huggingface.co/new-space

C. Give your Space a name and a license. Select "Gradio" as the Space SDK, and then choose "Public" if you are fine with everyone accessing your Space and the underlying code

D. Then you will find a page that provides you instructions on how to upload your files into the Git repository for that Space. You may also need to add a `requirements.txt` file to specify any Python package dependencies.

E. Once you have pushed your files, that's it! Spaces will automatically build your Gradio demo allowing you to share it with anyone, anywhere!

![GIF](https://huggingface.co/blog/assets/28_gradio-spaces/spaces-demo-finalized.gif)


You can even embed your Gradio demo on any website -- in a blog, a portfolio page, or even in a colab notebook, like I've done with a Pictionary sketch recognition model below:

In [15]:
from IPython.display import IFrame

IFrame(
    src="https://hf.space/gradioiframe/stabilityai/stable-diffusion/+",
    width=1000,
    height=800,
)